In [6]:
'''
COTR demo for human face
We use an off-the-shelf face landmarks detector: https://github.com/1adrianb/face-alignment
'''
import argparse
import os
import time
import copy

import cv2
import numpy as np
import torch
import imageio
import matplotlib.pyplot as plt
# import torchprof

from COTR.utils import utils, debug_utils
from COTR.utils.stopwatch import StopWatch

from COTR.models import build_model
from COTR.options.options import *
from COTR.options.options_utils import *
from COTR.inference.inference_helper import triangulate_corr
from COTR.inference.sparse_engine import SparseEngine

from pytorch_memlab import MemReporter
from torchinfo import summary

utils.fix_randomness(0)
torch.set_grad_enabled(False)


def main(opt):
    model = build_model(opt)
    model = model.cuda()
    weights = torch.load(opt.load_weights_path, map_location='cpu')['model_state_dict']
    utils.safe_load_weights(model, weights)

    # eval(): switch to inference mode
    model = model.eval()
    
    return model


parser = argparse.ArgumentParser()
set_COTR_arguments(parser)
parser.add_argument('--out_dir', type=str, default=general_config['out'], help='out directory')
parser.add_argument('--load_weights', type=str, default=None, help='load a pretrained set of weights, you need to provide the model id')

args = ['--load_weights=default'] #MEMO:

opt = parser.parse_args(args)
opt.command = ' '.join(args)

layer_2_channels = {'layer1': 256,
                    'layer2': 512,
                    'layer3': 1024,
                    'layer4': 2048, }
opt.dim_feedforward = layer_2_channels[opt.layer]
if opt.load_weights:
    opt.load_weights_path = os.path.join(opt.out_dir, opt.load_weights, 'checkpoint.pth.tar')
print_opt(opt)
model = main(opt)

# get backbone
teacher_model = model.backbone[0].body

student_model = copy.deepcopy(teacher_model)


---------------------- OPTIONS ----------------------

                 backbone  resnet50
                  command  --load_weights=default
               dec_layers  6
                 dilation  False
          dim_feedforward  1024
                  dropout  0.1
               enc_layers  6
               hidden_dim  256
                    layer  layer3
             load_weights  default
        load_weights_path  ./out/default/checkpoint.pth.tar
                   nheads  8
                  out_dir  ./out
       position_embedding  lin_sine

----------------------------------------------------
using lin_sine as positional encoding
TR@ /COTR/COTR/models/backbone.py(149):train_backbone:False
freeze conv1.weight
freeze layer1.0.conv1.weight
freeze layer1.0.conv2.weight
freeze layer1.0.conv3.weight
freeze layer1.0.downsample.0.weight
freeze layer1.1.conv1.weight
freeze layer1.1.conv2.weight
freeze layer1.1.conv3.weight
freeze layer1.2.conv1.weight
freeze layer1.2.conv2.weight
freeze 

In [7]:
student_model.train(True)

print(teacher_model.training, student_model.training)

False True
